---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [56]:
import networkx
import pandas
import numpy
import pickle

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
)

from sklearn.feature_selection import (
    RFECV,
    SelectFromModel,
)
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    train_test_split,
    )

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [57]:
part_one_graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [58]:
def graph_identification():
    """Identifies the type of graph each of the graphs is

    Returns:
     list: string identifiers for the type of graph
    """
    graph_types = []
    for graph in part_one_graphs:
        path = networkx.average_shortest_path_length(graph)
        coefficient = networkx.average_clustering(graph)
        if path > 6:
            if coefficient < 0.5:
                graph_types.append("SW_L")
            else:
                raise Exception("unexpected type")
        else:
            if coefficient < 0.5:
                graph_types.append("PA")
            else:
                graph_types.append("SW_H")
    return graph_types

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [59]:
email = networkx.read_gpickle('email_prediction.txt')
print(networkx.info(email))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [60]:
class RandomForest(object):
    """builds the random forest

    Args:
     x_train(array): data to train on
     y_train(array): targets for training
     start (int): start value for number of estimators
     stop (int): upper value for range of estimators
     step (int): increment for range of estimators
     folds (int): K-folds for cross-validation    
    """
    def __init__(self, x_train, y_train,
                 start=10, stop=100, step=10, folds=10):
        self.x_train = x_train
        self.y_train = y_train
        self.start = start
        self.stop = stop
        self.step = step
        self.folds = folds
        self._parameters = None
        self._search = None
        self._model = None
        return

    @property
    def parameters(self):
        """parameters for the grid-search"""
        if self._parameters is None:
            self._parameters = dict(n_estimators=range(self.start,
                                                       self.stop,
                                                       self.step))
        return self._parameters

    @property
    def search(self):
        """fitted grid search to find hyper-parameters"""
        if self._search is None:
            self._search = GridSearchCV(RandomForestClassifier(),
                                        self.parameters,
                                        cv=StratifiedKFold(self.folds),
                                        scoring="roc_auc")
            self._search.fit(self.x_train, self.y_train)
        return self._search

    @property
    def model(self):
        """best model found by the grid search"""
        if self._model is None:
            self._model = self.search.best_estimator_
        return self._model

In [61]:
class DataLoader(object):
    """loads and transforms the data
    Args:
     estimators (int): number of trees to use for feature elimination
    """
    def __init__(self, estimators=10):
        self.estimators = estimators
        self._data = None
        self._dummies_data = None
        self._training_data = None
        self._prediction_data = None
        self._non_management = None
        self._y_train = None
        self._x_train = None
        self._x_predict = None
        self._scaler = None
        self._x_train_scaled = None
        self._x_predict_scaled = None
        self._eliminator = None
        self._x_train_reduced = None
        self._x_predict_reduced = None
        return

    @property
    def data(self):
        """The initial data"""
        if self._data is None:
            data = pandas.DataFrame(index=email.nodes())
            data["department"] = pandas.Series(networkx.get_node_attributes(email, "Department"))
            data["management"] = pandas.Series(networkx.get_node_attributes(email, "ManagementSalary"))
            data["clustering"] = pandas.Series(networkx.clustering(email))
            data["degree"] = pandas.Series(email.degree())
            data["degree_centrality"] = pandas.Series(networkx.degree_centrality(email))
            data["closeness_centrality"] = pandas.Series(networkx.closeness_centrality(email))
            data["betweenness_centrality"] = pandas.Series(networkx.betweenness_centrality(email))
            data["pagerank"] = pandas.Series(networkx.pagerank(email))
            _, authority = networkx.hits(email)
            data["authority"] = pandas.Series(authority)            
            self._data = data
        return self._data

    @property
    def dummies_data(self):
        """one-hot-encoded data"""
        if self._dummies_data is None:
            self._dummies_data = pandas.get_dummies(self.data, columns=["department"])
        return self._dummies_data

    @property
    def training_data(self):
        """data with management information"""
        if self._training_data is None:
            self._training_data = self.dummies_data[pandas.notnull(
                self.dummies_data.management)]
        return self._training_data

    @property
    def prediction_data(self):
        """data missing management information"""
        if self._prediction_data is None:
            self._prediction_data = self.dummies_data[pandas.isnull(
                self.dummies_data.management)]
            assert len(self._prediction_data) == 252
        return self._prediction_data

    @property
    def non_management(self):
        """list of columns minus management"""
        if self._non_management is None:
            self._non_management = [
                column for column in self.training_data.columns
                if column != "management"]
        return self._non_management

    @property
    def y_train(self):
        """target-data for training"""
        if self._y_train is None:
            self._y_train = self.training_data.management
        return self._y_train

    @property
    def x_train(self):
        """data for training"""
        if self._x_train is None:
            self._x_train = self.training_data[self.non_management]
        return self._x_train

    @property
    def x_predict(self):
        """set to make predictions"""
        if self._x_predict is None:
            self._x_predict = self.prediction_data[self.non_management]
        return self._x_predict

    @property
    def scaler(self):
        """standard scaler"""
        if self._scaler is None:
            self._scaler = StandardScaler()
        return self._scaler

    @property
    def x_train_scaled(self):
        """training data scaled to 1 std, 0 mean"""
        if self._x_train_scaled is None:
            self._x_train_scaled = self.scaler.fit_transform(self.x_train)
        return self._x_train_scaled

    @property
    def x_predict_scaled(self):
        """prediction data with mean 0, std 1

        The answer requires the index so this is a dataframe
        instead of an array

        Returns:
         pandas.DataFrame: scaled data with index preserved
        """
        if self._x_predict_scaled is None:
            self._x_predict_scaled = pandas.DataFrame(
                self.scaler.transform(self.x_predict),
                index=self.x_predict.index)
        return self._x_predict_scaled

    @property
    def eliminator(self):
        """recursive feature eliminator"""
        if self._eliminator is None:
            trees = ExtraTreesClassifier(n_estimators=10)
            self._eliminator = RFECV(estimator=trees, cv=StratifiedKFold(10), 
                                     scoring="roc_auc")
            self._eliminator.fit(self.x_train_scaled, self.y_train)
        return self._eliminator

    @property
    def x_train_reduced(self):
        """training data with features eliminated"""
        if self._x_train_reduced is None:
            self._x_train_reduced = self.eliminator.transform(
                self.x_train_scaled)
        return self._x_train_reduced

    @property
    def x_predict_reduced(self):
        """prediction data with features eliminated"""
        if self._x_predict_reduced is None:
            self._x_predict_reduced = pandas.DataFrame(
                self.eliminator.transform(self.x_predict_scaled),
                index=self.x_predict_scaled.index)
        return self._x_predict_reduced

In [62]:
def salary_predictions():
    data = DataLoader()
    forest = RandomForest(data.x_train_reduced, data.y_train)
    probabilities = forest.model.predict_proba(data.x_predict_reduced)
    return pandas.Series(probabilities[:, 1], index=data.x_predict_reduced.index)    

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [63]:
future_connections = pandas.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

            Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [64]:
class Futures(object):
    target = "Future Connection"
    data_file = "Future_Connections.csv"
    graph_file = "email_prediction.txt"
    networkx_data_index = 2
    folds = 10

In [65]:
class DataNames(object):
    resource_allocation = 'resource_allocation'
    jaccard = 'jaccard_coefficient'
    adamic = "adamic_adar"
    preferential = "preferential_attachment"

In [66]:
def add_networkx_data(adder, name, graph=email, frame=future_connections):
    """Adds networkx data to the frame

    The networkx link-prediction functions return generators of triples:
     (first-node, second-node, value)

    This will use the index of the frame that's passed in as the source of 
    node-pairs for the networkx function (called `ebunch` in the networkx
    documentation) and the add only the value we want back to the frame

    Args:
     adder: networkx function to call to get the new data
     name: column-name to add to the frame
     graph: networkx graph to pass to the function
     frame (pandas.DataFrame): frame with node-pairs as index to add data to
    """
    frame[name] = [output[Futures.networkx_data_index]
                   for output in adder(graph, frame.index)]
    return frame

In [67]:
future_connections = add_networkx_data(networkx.resource_allocation_index,
                                       DataNames.resource_allocation)

In [68]:
future_connections = add_networkx_data(networkx.jaccard_coefficient, DataNames.jaccard)

In [69]:
future_connections = add_networkx_data(networkx.adamic_adar_index, DataNames.adamic)

In [70]:
future_connections = add_networkx_data(networkx.preferential_attachment, DataNames.preferential)

In [71]:
prediction_set = future_connections[future_connections[Futures.target].isnull()]
training_set = future_connections[future_connections[Futures.target].notnull()]

In [72]:
assert len(prediction_set) + len(training_set) == len(future_connections)

In [73]:
non_target = [column for column in future_connections.columns
              if column != Futures.target]
x_train = training_set[non_target]
y_train = training_set[Futures.target]
x_predict = prediction_set[non_target]

In [75]:
assert all(x_train.columns == x_predict.columns)
assert len(x_train) == len(y_train)

In [76]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_predict_scaled = scaler.transform(x_predict)

x_train_frame = pandas.DataFrame(x_train_scaled, columns=x_train.columns)
x_predict_frame = pandas.DataFrame(x_predict_scaled, columns=x_predict.columns)

In [77]:
estimator = ExtraTreesClassifier()
estimator.fit(x_train_scaled, y_train)
selector = SelectFromModel(estimator, prefit=True)
x_train_trees_sfm = selector.transform(x_train_scaled)
x_predict_sfm = selector.transform(x_predict_scaled)

In [78]:
model = LogisticRegressionCV(n_jobs=-1, scoring='roc_auc', solver='liblinear',
                             cv=StratifiedKFold())
model.fit(x_train_trees_sfm, y_train)

LogisticRegressionCV(Cs=10, class_weight=None,
           cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=-1, penalty='l2',
           random_state=None, refit=True, scoring='roc_auc',
           solver='liblinear', tol=0.0001, verbose=0)

In [79]:
def new_connections_predictions():
    probabilities = model.predict_proba(x_predict_sfm)
    return pandas.Series(probabilities[:, 1], index=prediction_set.index)